<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/qf-workshop-2021/introduction-to-machine-learning/blob/main/1_notebooks/introduction-to-machine-learning.ipynb">
        <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Supervised Models: Decision Trees

## What is a decision tree?

Decision trees can be used for regression (continuous real-valued output, e.g. predicting the price of a house) or classification (categorical output, e.g. predicting email spam vs. no spam), but here we will focus on classification. A decision tree classifier is a binary tree where predictions are made by traversing the tree from root to leaf — at each node, we go left if a feature is less than a threshold, right otherwise. Finally, each leaf is associated with a class, which is the output of the predictor.

### Why Decision Trees?

- Decision tress often mimic the human level thinking so its so simple to understand the data and make some good interpretations.

- Decision trees actually make you see the logic for the data to interpret(not like black box algorithms like SVM,NN,etc..)

For example : if we are classifying bank loan application for a customer, the decision tree may look like this
Here we can see the logic how it is making the decision.

It’s simple and clear.
So what is the decision tree?

## Decision Trees for Classification

During model training on feature-target relationships, a tree is grown from a root (parent) node (all data containing feature-target relationships), which is then recursively split into child nodes (subset of the entire data) in a binary fashion. Each split is performed on a single feature in the parent node, at a desired threshold value of the feature. For instance, during each split of the parent node, we go to left node (with the corresponding subset of data) if a feature is less than the threshold, and right node otherwise. But how do we decide on the split?


### Gini Impurity

The cost function that determines the split of the parent node is called the Gini Impurity, which is basically a concept to quantify how homogeneous or “pure” a node is, with relation to the distribution of the targets in the node. A node is considered pure (G=0) if all training samples in the node belong to the same class, while a node with many training samples from many different classes will have a Gini Impurity close to 1.

\begin{equation}
G = 1 - \sum\limits_{k=1}^n p_k^2 = 1 - \sum\limits_{k=1}^n \left( \frac{m_k}{m} \right)^2 
\end{equation}

where $p_k$ is the fraction of samples belonging to class $k$, $n$ is the number of classes, $m$ are all the training samples in the node and $m_k$ are the training examples in each class.

For example if a node contains five samples, with two of class 1, two of class 2, one of class 3 and none of class 4, then

$$G = 1 - \left( \frac{2}{5} \right)^2 - \left( \frac{2}{5} \right)^2  - \left( \frac{1}{5} \right)^2  = 0.64$$

### CART algorithm

The training algorithm is a recursive algorithm called CART, short for Classification And Regression Trees. The Decision Tree grows in the direction of decreasing Gini Impurity, and the root node is the most impure. Hence, at each node the optimum feature, and the corresponding optimum threshold value, are searched and selected, such that the weighted average Gini Impurity of the 2 child nodes is the least possible. The recursion stops when the maximum depth, a hyperparameter, is reached, or when no split can lead to two children purer than their parent. Other hyperparameters can control this stopping criterion (crucial in practice to avoid overfitting), but we won’t cover them here.

#### Finding the optimal feature and threshold

The key to the CART algorithm is finding the optimal feature and threshold such that the Gini impurity is minimized. To do so, we try all possible splits and compute the resulting Gini impurities.